# Time Series analysis of HTTP Bro Log

In [1]:
%matplotlib notebook
import pandas as pd


Data input

In [3]:
http_df = pd.read_csv("resourcse/http.log", header=None, sep="\t", 
                      names=['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'trans_depth', 'method', 'host', 'uri', 'referrer', 
                             'user_agent', 'request_body_len', 'response_body_len', 'status_code', 'status_msg', 'info_code', 'info_msg', 'filename', 'tags', 
                             'username', 'password', 'proxied', 'orig_fuids', 'orig_mime_types', 'resp_fuids', 'resp_mime_types', 'sample']) 
#resources from http://www.secrepo.com/Datasets%20Description/Network/http.html
http_df.head()

FileNotFoundError: File b'resourcse/http.log' does not exist

Timestamp

In [ ]:
#clean up the timestamp column ts
from datetime import datetime
http_df['ts'] = [datetime.fromtimestamp(float(date)) for date in http_df['ts'].values]
http_df = http_df.set_index('ts')
http_df.head()
# code provided by https://github.com/sooshie/Security-Data-Analysis/blob/master/Lab_2/Lab_2-Solutions.ipynb

Graphing time series data

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16.0, 5.0)
df = http_df[['request_body_len','response_body_len']]
df.plot()
# code provided by https://github.com/sooshie/Security-Data-Analysis/blob/master/Lab_2/Lab_2-Solutions.ipynb

NameError: name 'http_df' is not defined

Resamping data

In [ ]:
resamp = df.resample("M", how=['mean', 'count', 'sum']) #"M" means month end frequency
resamp.plot(subplots=True)
resamp.plot()
# code provided by https://github.com/sooshie/Security-Data-Analysis/blob/master/Lab_2/Lab_2-Solutions.ipynb

Times Series Model-- ARMA(p,q)
https://en.wikipedia.org/wiki/Autoregressive–moving-average_model#ARMA_model
using ACF and PACF to determine p and q.
https://onlinecourses.science.psu.edu/stat510/node/62/
ACF and PACF
https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/
Python code for time series
https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/

Try modelling the sum

In [ ]:

# autocorrelation plot (acf)
from pandas.tools.plotting import autocorrelation_plot
from matplotlib import pyplot
autocorrelation_plot(resamp['response_body_len']['sum'])

According to the graph, we can get q = 2

In [ ]:
# pacf partial autocorrelation func
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(resamp['response_body_len']['sum']) 

too many value >1 or <-1, i don't think it is ar model.

Fit model and predict

In [ ]:
# MA 
from statsmodels.tsa.arima_model import ARMA
# fit model
model = ARMA(resamp['response_body_len']['sum'], order=(0, 2))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(resamp['response_body_len']['sum']), len(resamp['response_body_len']['sum']))
print(yhat)